In [47]:
# the only thing that needs to be detokenized is punctuation being separated by a space. And there being newline characters. (should these be replaced??)

from sacremoses import MosesTokenizer, MosesDetokenizer
import pandas as pd
md = MosesDetokenizer(lang='en')

for variant in ['test']:#, 'train']:
    cleaned = dict()
    
    for s_or_t in ['source', 'target']:
        f = variant+'.wp_'+s_or_t
    
        with open(f, 'r') as file:
            text = file.readlines()
        detokened = []
        for ind in range(0,len(text)):
            detokened.append(md.detokenize(text[ind:ind+1]).replace('<newline>', '\n'))
        assert len(text)==len(detokened), "text and detokened are not the same length!"
        cleaned[variant+'_'+s_or_t] = detokened
        
    pd.DataFrame.from_dict(cleaned).to_csv(variant+'_dataframe.csv', index=False)


In [65]:
df = pd.DataFrame.from_dict(cleaned)

In [66]:
df.head()

,test_source,test_target
0,[WP] Leonardo DiCaprio in a fit of rage begins...,The wet marble floor pressed on his cheek like...
1,[CW] Kill the writer in first-person narrative.,It's been three days since my boyfriend pissed...
2,[EU] Sean Bean has a hard time leaving his rol...,"George fled. Not terrifically fast, not at his..."
3,[WP] A kid doodling in a math class accidental...,It was dark and Levi was pretty sure he was ly...
4,[WP] As a Space marine you have an allowance o...,``Hi dad! ''\n \n The words snap me from my th...


In [109]:
df.shape

(15138, 4)

In [108]:
# how many unique prompts are there? 
df['test_source'].unique().shape

(5478,)

In [91]:
'[' in '[WP]'

True

In [112]:
'''starters = []
for e in df['test_source'].str[:30].unique().tolist():
    if '[' in e:
        starters.append(e)
starts = []
for s in starters:
    starts.append(s.split(']')[0][1:])
list(set(starts))'''

"starters = []\nfor e in df['test_source'].str[:30].unique().tolist():\n    if '[' in e:\n        starters.append(e)\nstarts = []\nfor s in starters:\n    starts.append(s.split(']')[0][1:])\nlist(set(starts))"

In [114]:
df['test_source'][df['test_source'].str.contains('Wiring Promp')]

1748     [Wiring Prompt] A person who commits a crime f...
7691     [Wiring Prompt] A person who commits a crime f...
11362    [Wiring Prompt] A person who commits a crime f...
13895    [Wiring Prompt] A person who commits a crime f...
14969    [Wiring Prompt] A person who commits a crime f...
Name: test_source, dtype: object

In [118]:
import numpy as np
df['test_target_200'] = df['test_target'].apply(lambda x: ' '.join(x.split(' ')[:200]))
df['Prompt'] = df['test_source'] + " \n "+df['test_target_200'] 
df.head()

,test_source,test_target,test_target_200,Prompt
0,[WP] Leonardo DiCaprio in a fit of rage begins...,The wet marble floor pressed on his cheek like...,The wet marble floor pressed on his cheek like...,[WP] Leonardo DiCaprio in a fit of rage begins...
1,[CW] Kill the writer in first-person narrative.,It's been three days since my boyfriend pissed...,It's been three days since my boyfriend pissed...,[CW] Kill the writer in first-person narrative...
2,[EU] Sean Bean has a hard time leaving his rol...,"George fled. Not terrifically fast, not at his...","George fled. Not terrifically fast, not at his...",[EU] Sean Bean has a hard time leaving his rol...
3,[WP] A kid doodling in a math class accidental...,It was dark and Levi was pretty sure he was ly...,It was dark and Levi was pretty sure he was ly...,[WP] A kid doodling in a math class accidental...
4,[WP] As a Space marine you have an allowance o...,``Hi dad! ''\n \n The words snap me from my th...,``Hi dad! ''\n \n The words snap me from my th...,[WP] As a Space marine you have an allowance o...


In [119]:
df['Prompt'] = df['Prompt'].apply(lambda x: x.replace('`` ', ' ``'))

In [120]:
pd.options.display.max_rows = 999
df['Prompt'].head(1000)

0      [WP] Leonardo DiCaprio in a fit of rage begins...
1      [CW] Kill the writer in first-person narrative...
2      [EU] Sean Bean has a hard time leaving his rol...
3      [WP] A kid doodling in a math class accidental...
4      [WP] As a Space marine you have an allowance o...
5      [TT] ``Shut the dog up. '' \n “Shut the dog up...
6      [WP] A man finally discovers his superpower......
7      [WP] Reddit buys the moon \n Dickbutt... That ...
8      [WP] You have a very shitty type of precogniti...
9      [WP] ``Before we allow humankind to live among...
10     [WP] A magical mirror shows your reflection an...
11     [WP] A person with a high school education get...
12     [WP] Everyone has a reaper. The further away i...
13     [WP] A person with a high school education get...
14     [WP] After brushing your teeth in the morning ...
15     [WP] While searching the site of a disaster, y...
16     [WP] While walking home from work one day, som...
17     [WP] Fabricate an inspir

In [121]:
df.to_csv(variant+'_dataframe.csv', index=False)